In [162]:
import pandas as pd
from transformers import BertTokenizer
import json
from tqdm import tqdm
from konlpy.tag import Mecab
mecab = Mecab()

In [163]:
# pd.set_option('display.max_colwidth', None) # 전체 텍스트 출력

In [164]:
# df_pred1 = pd.read_csv('NER/predict/electra_final/predict_NER_electra_final_ids_rev_21.11.24-1.csv')
# df_pred = pd.read_csv('NER/predict/electra_final/predict_NER_electra_final_ids_rev_22.2.24-1.csv')
df_pred = pd.read_csv('NER/predict/BiLSTM/predict_NER_BiLSTM_ids_rev_22.3.2-5.csv')

In [165]:
# tokenizer = BertTokenizer.from_pretrained('vocab.txt', do_lower_case=False)
tokenizer = BertTokenizer.from_pretrained('../nscc_mecab_base_final/vocab.txt', do_lower_case=False) # 최신 vocab

/home/ubuntu/anaconda3/envs/pytorch1.4_p36/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:1646: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


In [166]:
# with open("ner_to_index_rev_1118.json", 'rb') as f:
#         ner_to_index = json.load(f)
# index_to_ner = {}
# for key, value in ner_to_index.items():
#     index_to_ner[value] = key
    
with open("ner_to_index_rev_220214.json", 'rb') as f:
        ner_to_index = json.load(f)
index_to_ner = {}
for key, value in ner_to_index.items():
    index_to_ner[value] = key

In [6]:
# enc_input : 인코딩된 input 데이터, pred : 예측 데이터
# return 값은 B, I 라벨링된 텍스트 데이터
def pred_to_BI_text(enc_input, pred):
    idx = 0
    label_texts = ''
    for i in range(len(pred)):
        if i >= len(enc_input):
            label_texts += tokenizer.decode(enc_input[prev_idx:idx]).replace('#', '')
            break
        num = int(pred[i])
        if (num) > 4:
            label_texts += tokenizer.decode(enc_input[idx:i]).replace('#', '') + ' '
            label_text = "<{}:{}>".format(tokenizer.decode(enc_input[i]).replace(' ', '').replace('#', ''), index_to_ner[num])
            label_texts += label_text + ' '
            idx = i+1
            
    label_texts = ' '.join(label_texts.split())
    return label_texts

In [109]:
# enc_input : 인코딩된 input 데이터, pred : 예측 데이터
# return 값은 코드 라벨링된 텍스트 데이터
def pred_to_text(enc_input, pred): 
    idx = 0
    prev_idx = 0
    label_texts = ''
    B_chk = False
    while idx < len(pred):
        if idx >= len(enc_input):
            label_texts += tokenizer.decode(enc_input[prev_idx:idx]).replace('#', '')
            break
        num = int(pred[idx])
        if num > 4: # 태그인 경우
            label_texts += tokenizer.decode(enc_input[prev_idx:idx]).replace('#', '')  # 태그가 등장하기 전 인덱스 값들을 텍스트로 변환
            text = ''
            while int(pred[idx]) > 4: # 다음 인덱스들이 태그가 아닐 때까지 반복
                if idx >= len(enc_input):
                    break
                if not B_chk: # B_chk 가 False 이면 (B 태그가 등장하지 않았을 때)
                    if 'B▁' in index_to_ner[int(pred[idx])]: # B태그라면
                        code = index_to_ner[int(pred[idx])][2:] # 코드를 저장
                        text += ' ' + tokenizer.decode(int(enc_input[idx])).replace('#', '').replace(' ', '') # 인덱스 값을 텍스트로 변환
                        B_chk = True # 이전 인덱스 값이 B태그라는 표시
                        idx += 1 # 다음 인덱스 확인
                    elif 'I▁' in index_to_ner[int(pred[idx])]: # I태그라면
                        idx += 1 # 다음 인덱스 확인
                        B_chk = False # 이전 인덱스 값이 B태그가 아니라는 표시
                        break
                    else: # B, I 태그가 아니라면
                        text += ' ' + tokenizer.decode(int(enc_input[idx])).replace('#', '').replace(' ', '') # 인덱스 값을 텍스트로 변환
                        B_chk = False # 이전 인덱스 값이 B태그가 아니라는 표시
                        break
                elif B_chk: # B_chk 가 True 이면 (이전에 B 태그가 등장했을 때)
                    if 'I▁' in index_to_ner[int(pred[idx])]: # I 태그라면
                        text += ' ' + tokenizer.decode(int(enc_input[idx])).replace('#', '').replace(' ', '') # 인덱스 값을 텍스트로 변환
                        idx += 1 # 다음 인덱스 확인
#                         B_chk = False # 이전 인덱스 값이 B태그가 아니라는 표시
                    elif 'B▁' in index_to_ner[int(pred[idx])]:
                        B_chk = False # 이전 인덱스 값이 B태그가 아니라는 표시
                        break
                    else: # B, I 태그가 아니라면
                        B_chk = False # # 이전 인덱스 값이 B태그가 아니라는 표시
                        break
            if len(text) > 0:
                try:
                    label_text = "<{}:{}>".format(text.strip(), code)
                except:
                    print(pred)
                    print(index_to_ner[int(pred[idx])])
                    break
                label_texts += ' ' + label_text + ' '
            prev_idx = idx
        else: # 태그가 아닌 경우
            idx += 1
            label_texts += ' ' + tokenizer.decode(enc_input[prev_idx:idx]).replace('#', '') + ' '
            prev_idx = idx
    
    label_texts = ' '.join(label_texts.split())
    return label_texts

In [167]:
df_pred

,input,encode_input,target,predict
0,본 논문은 팰러닉의 『파이트 클럽』이 함의하는 정치적 의의를 재고하려는 목적을 가진...,[ 202 1669 6069 2443 3268 1261 907 24...,"[0, 4, 4, 4, 4, 4, 4, 4, 4, 169, 4, 4, 4, 4, 4...",[ 0 4 4 4 4 4 4 4 4 169 4 ...
1,[발명의 명칭] 하이브리드 차량용 변속장치HYBRID POWERTRAIN 본 발명은...,[ 202 260 1586 4077 2464 13666 261 95...,"[0, 4, 4, 4, 4, 4, 4, 155, 156, 4, 4, 4, 4, 4,...",[ 0 4 4 4 4 4 4 155 156 4 4 ...
2,목적: 급성 전골수구백혈병 환자에서 all-trans retinoic acid (A...,[ 202 6165 228 542 1916 2557 3798 36...,"[0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",[ 0 4 4 4 4 4 4 4 4 4 4 ...
3,[발명의 명칭] TRAP1 유전자를 이용한 미토콘드리아 기능 장애 치료제 스크리닝 ...,[ 202 260 1586 4077 2464 13666 261 108...,"[0, 4, 4, 4, 4, 4, 4, 219, 4, 4, 4, 4, 4, 4, 1...",[ 0 4 4 4 4 4 4 219 4 4 4 ...
4,"논문은 존 버거의 『다른 방식으로 보기』 3장을 읽고,﻿‘대상화’를 탐구하는 작업이...",[ 202 6069 2443 2600 1623 4016 2464 2...,"[0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 231, 232, 4,...",[ 0 4 4 4 4 4 4 4 4 4 4 ...
...,...,...,...,...
29995,"본 발명은 내부 공간을 형성하며, 전면에 복수의 엘이디가 어레이된 엘이디 모듈을 포...",[ 202 1669 1586 4077 2443 6318 5998 24...,"[0, 4, 4, 4, 4, 4, 4, 4, 219, 4, 4, 4, 4, 4, 2...",[ 0 4 4 4 4 4 4 4 219 4 4 ...
29996,백두산 화산 마그마 특성 및 거동 규명 백두산 화산층서도 작성 방안 도출 및 예비조...,[ 202 1604 3757 3691 14028 12127 4097 56...,"[0, 281, 282, 282, 282, 4, 4, 4, 4, 4, 4, 281,...",[ 0 4 4 4 4 4 4 4 4 4 4 ...
29997,하이브리드 로켓의 연소실험에서 관찰된 500 Hz 대역의 연소압력과 연소반응 변동이...,[ 202 9509 1303 4255 2464 7532 7358 55...,"[0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 71, 4, 4,...",[ 0 4 4 4 4 4 4 4 4 4 4 ...
29998,[발명의 명칭] 폭력 상황을 검출하는 장치 및 방법APPARATUS AND METH...,[ 202 260 1586 4077 2464 13666 261 122...,"[0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",[ 0 4 4 4 4 4 4 4 4 4 4 ...


In [168]:
df_pred['input'][21432]

'[발명의 명칭] 카메라 모듈Camera Module 본 발명의 실시예는 렌즈 배럴의 구조가 개선된 카메라 모듈에 관한 것이다.  각종 휴대단말기의 보급이 널리 일반화되고, 무선 인터넷 서비스가 상용화됨에 따라 휴대단말기와 관련된 소비자들의 요구도 다양화되고 있는바, 이에 따라 다양한 종류의 부가장치들이 휴대단말기에 장착되고 있다. 그 중에서 피사체를 사진이나 동영상으로 촬영하여 그 이미지데이터를 저장한 후 필요에 따라 이를 편집 및 전송할 수 있는 대표적인 것이 카메라 모듈이다. 근래 노트형 퍼스널 컴퓨터, 카메라 폰, PDA, 스마트, 토이(toy) 등의 다종다양한 멀티미디어 분야, 나아가서는 감시 카메라나 비디오 테이프 레코더의 정보단말 등의 화상입력기기용으로 소형의 카메라 모듈의 수요가 높아지고 있다. 이러한 카메라 모듈에 적용되는 센서의 픽셀사이즈가 소형화되고 모듈의 높이가 낮아짐에 따라, 이물질의 유입에 대한 관리와 렌즈 광축의 틸트(tilt)나 시프트(Shift) 등의 카메라 모듈의 기구적 신뢰성을 확보하기 위한 마운트 구조의 개발이 매우 중요한 화두로 떠오르고 있다. 렌즈 배럴과 보빈이 나사산 결합을 하는 카메라 모듈은 포커싱 조정작업시 수나사부와 암나사부간의 마찰에 의하여 이물이 발생되고, 이때 발생된 이물에 의해서 화상품질이 저하될 수 있기 때문에, 무나사 타입의 카메라 모듈에 대한 개발이 이루어지고 있다. 그런데, 무나사 타입의 렌즈 배럴을 갖는 카메라 모듈을 제작하는 과정에서 랜즈 배럴 조립을 위헤 에폭시 등을 사용함에 있어 렌즈 배럴과 보빈 사이에 침투된 에폭시가 하측으로 계속 이동하여 화상영역으로 전이됨으로써 이물 불량이 유발될 수 있다.  [발명의 개요]  [해결하려는 과제] 본 발명의 실시예는 무나사산 결합하는 렌즈 배럴을 포함하는 카메라 모듈의 신뢰성을 향상시킨다.  [과제의 해결 수단] 실시예는 이미지 센서가 실장 되는 인쇄회로기판과, 상기 인쇄회로기판의 상측에 설치되며, 상기 이미지 센서와 대응되는 위치에 배치되는 렌즈부와, 

In [170]:
# tokenizer.decode(' '.join(df_pred['encode_input'][1234][1:-1].replace(',', '').split()))
tokenizer.decode(df_pred['encode_input'][1111][1:-1].replace(',', '').split())

'[CLS] 경남지역 일반농경지 토양 물리 성 분석 DB 구축 연도 별 정점조사 대상지 2017 밭, 2018 과 수원, 2019 논, 2020 시설재배지 조사 점수 매년 40 지점 조사항목 토성, 용적 밀도, 삼상 등 경남지역 농업용수 수질 분석 DB 구축 조사 대상 하천수 39 지점 4, 7, 10 월, 지하수 20 지점 4, 7 월 하천수 법적기준 6 항목 pH, DO, BOD, CODMn, TP, SS, 기타 7 항목 EC, TN, NH 4 N, Ca, K, Mg, Na, TOC 지하수 법적기준 7 항목 pH, NO 3 N, Cl, Cd, As, Hg, Pb, 기타 8 항목 EC, Ca, K, Mg, Na, SO 42, TN, TP [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [171]:
# enc_input = tokenizer.encode(df_pred['input'].iloc[35])
# enc_input = tokenizer.encode(' '.join(mecab.morphs(df_pred['input'].iloc[35]))) # list로 한 단어를 넣게 되면 encode가 UNK로 나온다!, list를 다 str으로 바꿔줘야된다!
num = 5555
enc_input = df_pred['encode_input'].iloc[num][1:-1].replace(',', '').split()
target = df_pred['target'].iloc[num][1:-1].split(', ')
pred = ' '.join(df_pred['predict'][0][1:-1].strip().replace('\n', '').split()).split(' ')

In [172]:
df_pred['input'][5555].strip()

'[발명의 명칭] 광학 필름, 광학 필름의 제조 방법, 반사방지 필름, 편광판 및 화상 표시 장치 OPTICAL FILM, METHOD OF PRODUCING OPTICAL FILM, ANTIREFLECTIVE FILM, POLARIZING PLATE AND IMAGE DISPLAY DEVICE 본 발명은 광학 필름, 광학 필름의 제조 방법, 반사방지 필름, 편광판 및 화상 표시 장치에 관한 것이다. 보다 상세하게는, 본 발명은 셀룰로오스 아실레이트 기재 상에 대전방지성 하드코트층을 포함하고, 대전방지성, 경도, 및 고온 및 고습 환경에 노출된 후의 대전방지성이 우수한 광학 필름, 및 상기 광학 필름을 함유하는 반사방지 필름, 편광판 및 화상 표시 장치에 관한 것이다.  화상 표시 장치, 예를 들어 음극선관 표시 장치 (CRT), 플라즈마 디스플레이 패널 (PDP), 전계발광 디스플레이 (ELD), 진공 형광 디스플레이 (VFD), 전계 방출 디스플레이 (FED) 또는 액정 표시 장치에서는, 파손이나 스크래치를 방지하고, 표시면에의 먼지 등의 부착에 의한 가시성의 저하 또는 정전기에 의한 표시 장치의 파괴를 방지하기 위해서, 투명하고, 대전방지성과 하드코트성을 갖는 광학 필름을 제공하는 것이 적합하다. 특히, 화상 표시 장치의 최근의 경향, 예를 들어 고품위화, 대형화, 박층화 또는 장수명화에 대응하여, 화상 표시 장치에 적용되는 광학 필름에 대해서도, 하드코트성 및 대전방지성에 더해 양호한 기타 성능, 및 고 내구성을 가지는 것이 요구되고 있다. 특히, 액정 표시 장치에서는, 하드코트성과 대전방지성의 양호한 양립성을 갖는 광학 필름을 편광판용 보호 필름으로서 사용하는 것이 적합하며, 이 편광판 형태의 편광자와의 양호한 밀착성을 유지하기 위해서, 광학 필름의 기재로서는 일반적으로 셀룰로오스 아실레이트 필름을 사용한다. 상기의 관점으로부터, 셀룰로오스 아실레이트 필름은 고 투명성 (저 헤이즈), 양호한 투습성 및 양호한 광학 성능을 가지는 것이 필요하며, 지금까

In [173]:
pred_to_text(enc_input, target)

'[CLS] [ 발 명 의 명칭 ] 광학 필름 , 광학 필름 의 제조 방법 , <반사 방지 필름:ED10> , 편 광 판 및 화상 표시 장치 OP T IC AL F IL M , ME T HO D O F PR OD UC IN G OP T IC AL F IL M , AN T IR E F L EC T IV E F IL M , <PO:EE01> LA RI Z IN G <PL:NB06> AT E AN D IM AG E D IS P LA Y D EV IC E 본 발 명 은 광학 필름 , 광학 필름 의 제조 방법 , <반사 방지 필름:ED10> , 편 광 판 및 화상 표시 장치 에 관한 것 이 다 . 보다 상세 하 게 는 , 본 발 명 은 셀룰 로오스 아 실 레이트 <기재:LA05> 상 에 <대전 방지 성:EC05> 하드 코 트 층 을 <포함:NA05> 하 고 , <대전 방지 성:EC05> , 경도 , 및 고온 및 고 습 <환경 에:ED07> 노출 된 후 의 <대전 방지 성:EC05> 이 우수 한 광학 필름 , 및 상기 광학 필름 을 함유 하 는 <반사 방지 필름:ED10> , 편 광 판 및 화상 표시 장치 에 관한 것 이 다 . 화상 표시 장치 , 예 를 들 어 음극 선 관 표시 장치 ( <CR T:LA01> ) , <플라즈마 디스플레이:ED10> 패널 ( <PD:LC02> P ) , 전 계 발 광 디스플레이 ( EL D ) , 진공 형광 디스플레이 ( V FD ) , 전 계 방출 디스플레이 ( F ED ) 또는 [SEP]'

In [174]:
pred_to_text(enc_input, pred)

'[CLS] [ 발 명 의 명칭 ] 광학 필름 <,:LA03> 광학 필름 의 제조 방법 , 반사 방지 필름 , 편 광 판 <및:EE02> 화상 표시 장치 OP T IC <AL:EE01> F IL M , ME T HO D O F PR OD UC IN G <OP:NC03> T IC AL F IL M , AN T IR E F L EC T IV E <F:EE01> IL M , PO LA RI Z IN G PL AT E AN D IM AG E D IS <P:EE02> LA Y D EV IC E 본 발 명 은 광학 필름 , 광학 필름 의 제조 방법 , 반사 방지 필름 , 편 광 판 및 화상 표시 장치 에 관한 것 이 다 . 보다 상세 하 게 는 , 본 발 명 은 셀룰 로오스 아 실 레이트 기재 상 에 대전 방지 성 <하드:EE02> 코 트 층 을 포함 하 고 , 대전 방지 성 , 경도 , 및 고온 및 고 습 환경 에 노출 된 후 의 대전 방지 성 이 우수 한 광학 필름 , 및 상기 광학 필름 을 함유 하 는 반사 <방지:LC01> 필름 , 편 광 판 및 화상 표시 장치 에 관한 것 이 다 . 화상 표시 장치 , 예 를 들 어 음극 선 관 표시 장치 ( CR T ) , 플라즈마 <디스플레이:EA07> 패널 ( PD P ) <,:EE01> 전 계 발 광 디스플레이 ( EL D ) , 진공 형광 디스플레이 ( V FD ) , 전 계 방출 디스플레이 ( F ED ) 또는 [SEP]'

In [175]:
for t, p in zip(target, pred):
    print(t, p)

0 0
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 169
4 4
4 4
4 4
4 4
4 4
4 4
81 4
82 4
82 4
4 4
4 4
4 4
4 4
4 87
4 4
4 4
4 4
4 4
4 4
4 4
4 85
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 269
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 85
4 4
4 4
4 4
85 4
4 4
4 4
4 4
4 4
4 4
263 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 87
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
81 4
82 4
82 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
173 4
4 4
4 4
55 4
56 4
56 4
4 87
4 4
4 4
4 4
4 4
251 4
4 4
4 4
4 4
55 4
56 4
56 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
75 4
76 4
4 4
4 4
4 4
4 4
55 4
56 4
56 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
81 4
82 219
82 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
165 4
166 4
4 4
4 4
81 4
82 17
4 4
4 4
221 4
4 4
4 4
4 85
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4
4 4


In [180]:
num = 8447
enc_input = df_pred['encode_input'].iloc[num][1:-1].replace(',', '').split()
target = df_pred['target'].iloc[num][1:-1].split(', ')
pred = ' '.join(df_pred['predict'][0][1:-1].strip().replace('\n', '').split()).split(' ')
print(pred_to_text(enc_input, target))
print(pred_to_text(enc_input, pred))

[CLS] <단일 세포 전사 체 시퀀 싱:LC01> 통합 분석 방법론 개발 을 위해 본 연구 는 우선 종양 을 세포 로 <구성:LC04> 된 집단 으로 보 는 새로운 관점 에서 진화 과정 분석 을 수행 하 며 이 어 <전사 체 시퀀 싱:LC02> 에서 얻 을 수 있 는 주요 정보 인 <발현 양:LA07> , <선택 적 이 어 맞추 기:LA02> , 편집 , <유전자 융합:LA07> 등 에 이 를 적용 하 여 각각 단일 세포 특화 된 새로운 방법론 을 개발 하 고자 한다 . 따라서 , 세부 목표 는 진화 과정 분석 <발현 양:LA07> 분석 <선택 적 이 어 맞추 기:LA02> 유전자 편집 유전자 융합 의 다섯 단계 로 이루어진다 . 우선 최근 개발 된 전사 체 로부터 유전자 형 정보 를 얻 어 내 는 분석 기 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] <[PAD]:ED10> [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] <[PAD]:LB02> [PAD] [PAD] <[PAD] [PAD]:LA07> [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] <[PAD]:EI03> [PAD] 

In [161]:
# enc_input = tokenizer.encode(df_pred['input'].iloc[35])
# enc_input = tokenizer.encode(' '.join(mecab.morphs(df_pred['input'].iloc[35]))) # list로 한 단어를 넣게 되면 encode가 UNK로 나온다!, list를 다 str으로 바꿔줘야된다!
num = 14
enc_input = df_pred['encode_input'].iloc[num][1:-1].replace(',', '').split()
target = df_pred['target'].iloc[num][1:-1].split(', ')
pred = ' '.join(df_pred['predict'][0][1:-1].strip().replace('\n', '').split()).split(' ')
print(df_pred['input'][num].strip())
pred_to_text(enc_input, target)

기술개발내용 최종목표 주관연구기업 농업회사법인 주식회사 삼흥 아로니아 추출물의 원료 및 건강기능식품 소재 제조공정 표준화 아로니아 안토시아닌 함량확인 및 지표성분 정량 및 1차 시제품 제작 아로니아 추출물의 전립선 비대증 효능을 가지는 시제품 2의 개발 협동연구기관 경남과학기술대학교 아로니아 추출물 제조 및 아로니아의 in vitro 전립선 비대 개선 효능 평가 아로니아로부터 기능성 성분의 분리 및 규명과 전립선비대 개선 기능성소재의 발굴 추출조건 최적화 및 기능성 복합물의 제조, 아로니아 추출물의 분석조건 확립 협동연구기관 안전성평가연구소 아로니아 추출물의 최종시료 선정을 위한 1차 in vivo 평가 활성 분획물 및 기능성 화합물의 2차 in vivo 실험 및 기능성 평가 자료의 확보 아로니아 추출물의 전립선 비대증 완화 효능 기전 연구


'[CLS] <기술 개발:EB02> 내용 최종목표 주관 연구 기업 농업 회사 법 인 <주식 회사:EF06> 삼 흥 <아 로 니아 추출물:LA09> 의 원료 및 <건강기능:LB17> 식품 소재 <제조공정 표준 화:LB16> 아 로 니아 안 토 시아 닌 함량 <확인:EE01> 및 지표 성분 정량 및 1 차 시제품 제작 아 로 니아 추출물 의 전 립 선 비대 증 효능 을 <가지:NC03> 는 시제품 2 의 개발 협동 연구 기관 경 남 과학기술 대학교 아 로 니아 추출물 제조 및 아 로 니아 의 <in:LA06> vitro 전 립 선 비대 개선 효능 평가 아 로 니아 로부터 <기능 성 성분:LB02> 의 분리 및 규명 과 전 립 선 비 대 개선 기능 성 소재 의 발굴 추출 조건 최적 화 및 기능 성 복합 물 의 제조 , 아 로 니아 추출물 의 분석 조건 <확립:LA03> 협동 연구 기관 안전성 평가 연구소 아 로 니아 추출물 의 최종 시료 <선정:LB07> 을 위한 1 차 <in:LA06> vivo 평가 활성 분획물 및 기능 성 화합물 의 2 차 <in:LA06> vivo 실험 및 기능 성 평가 자료 의 <확보:LB01> 아 로 니아 추출물 의 전 립 선 비대 증 완화 효능 기전 연구 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [55]:
target

['0',
 '4',
 '4',
 '4',
 '4',
 '4',
 '75',
 '76',
 '76',
 '121',
 '122',
 '122',
 '122',
 '122',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '255',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '183',
 '4',
 '4',
 '4',
 '121',
 '122',
 '122',
 '122',
 '122',
 '4',
 '75',
 '76',
 '76',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '265',
 '266',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '169',
 '4',
 '4',
 '81',
 '4',
 '4',
 '4',
 '4',
 '121',
 '122',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '87',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '193',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '

In [56]:
tokenizer.tokenize(df_pred['input'][num])

['100',
 '##배',
 '##율',
 '이상',
 '##의',
 '고',
 '##배',
 '##율',
 '집',
 '##광',
 '##형',
 '태양전지',
 '##모듈',
 '##을',
 '상용',
 '##화',
 '##하',
 '##기',
 '위해',
 '모듈',
 '##화',
 '##된',
 '상태',
 '##의',
 '어',
 '##셈',
 '##블',
 '##리',
 '파트',
 '##가',
 '대량',
 '##생산',
 '##모드',
 '##에',
 '##서',
 '생산',
 '##가능',
 '##한',
 '부품',
 '##으',
 '##로',
 '확보',
 '##되',
 '##어',
 '##야',
 '함',
 '집',
 '##광',
 '##형',
 '태양전지',
 '##모듈',
 '##은',
 '고',
 '##배',
 '##율',
 '집',
 '##광',
 '##을',
 '위해',
 '프',
 '##라이',
 '##머',
 '##리',
 '집',
 '##광',
 '##렌즈',
 '##와',
 '세',
 '##컨',
 '##더',
 '##리',
 '보정',
 '##렌즈',
 '##의',
 '사용',
 '##이',
 '필수',
 '##적',
 '##이',
 '##며',
 '이',
 '##를',
 '위해',
 '렌즈',
 '##의',
 '설계',
 '##와',
 '소재',
 '##에',
 '대한',
 '정확',
 '##한',
 '활용',
 '##방법',
 '##이',
 '정립',
 '대량',
 '##생산',
 '##과',
 '상업',
 '##적',
 '활용',
 '##을',
 '위해',
 'SO',
 '##G',
 '금형',
 '##을',
 '통한',
 '제조공정',
 '##을',
 '확립',
 '##하',
 '##여',
 '기존',
 '##의',
 '집',
 '##광',
 '##형',
 '태양전지',
 '##모듈',
 '##의',
 '문제점',
 '##으',
 '##로',
 '부각',
 '##되',
 '##었',
 '##던',
 '렌즈',
 '

In [57]:
for a, b in zip(target[1:-1], tokenizer.tokenize(df_pred['input'][num])):
    print(a, b)

4 100
4 ##배
4 ##율
4 이상
4 ##의
75 고
76 ##배
76 ##율
121 집
122 ##광
122 ##형
122 태양전지
122 ##모듈
4 ##을
4 상용
4 ##화
4 ##하
4 ##기
4 위해
4 모듈
4 ##화
4 ##된
255 상태
4 ##의
4 어
4 ##셈
4 ##블
4 ##리
4 파트
4 ##가
4 대량
4 ##생산
4 ##모드
4 ##에
4 ##서
4 생산
4 ##가능
4 ##한
4 부품
183 ##으
4 ##로
4 확보
4 ##되
121 ##어
122 ##야
122 함
122 집
122 ##광
4 ##형
75 태양전지
76 ##모듈
76 ##은
4 고
4 ##배
4 ##율
4 집
4 ##광
4 ##을
4 위해
4 프
4 ##라이
4 ##머
4 ##리
4 집
4 ##광
4 ##렌즈
4 ##와
4 세
4 ##컨
4 ##더
4 ##리
4 보정
4 ##렌즈
4 ##의
4 사용
4 ##이
4 필수
4 ##적
4 ##이
4 ##며
4 이
4 ##를
4 위해
4 렌즈
4 ##의
4 설계
4 ##와
265 소재
266 ##에
4 대한
4 정확
4 ##한
4 활용
4 ##방법
4 ##이
4 정립
4 대량
4 ##생산
4 ##과
4 상업
4 ##적
4 활용
4 ##을
4 위해
4 SO
4 ##G
4 금형
4 ##을
169 통한
4 제조공정
4 ##을
81 확립
4 ##하
4 ##여
4 기존
4 ##의
121 집
122 ##광
4 ##형
4 태양전지
4 ##모듈
4 ##의
4 문제점
4 ##으
4 ##로
4 부각
4 ##되
4 ##었
4 ##던
4 렌즈
4 ##활용
87 ##문제
4 ,
4 실
4 ##링
4 문제
4 등
4 ##을
4 극복
4 ##하
4 ##여
193 상업
4 ##적
4 이용
4 ##의
4 확대
4 ##와
4 상품
4 ##화
4 ##를
4 적극
1 추진
2 ##하
2 ##고
2 ##자
2 함


In [33]:
# enc_input = tokenizer.encode(df_pred['input'].iloc[35])
# enc_input = tokenizer.encode(' '.join(mecab.morphs(df_pred['input'].iloc[35]))) # list로 한 단어를 넣게 되면 encode가 UNK로 나온다!, list를 다 str으로 바꿔줘야된다!
enc_input = df_pred['encode_input'].iloc[num][1:-1].replace(',', '').split()
pred = df_pred['predict'].iloc[num][1:-1].replace(',', '').split()
(pred_to_text(enc_input, pred))

'[CLS] 1 . 혈관 <형성:LC01> 과정 에서 의 <유비:LA01> 퀴 틴 단백질 의 역할 <규명:OB02> 을 위해 혈관 세포 내 에서 발현 되 는 <유비:LA01> 퀴 틴 연결 효소 E 를 정량 적 으로 비교 분석 하 고 , 그 중 에서 선택 한 E 2 들 의 혈관 <형성:LC01> 과정 에서 역할 을 <규명:OB02> 함 . 2 . <TL:LC01> R signaling 에 의한 <연골퇴행:LA01> 조절 연구 를 진행 하 기 위해 연골 세포 에서 의 <TL:LC01> R signaling <관련유전자:LC02> 의 기능 을 <변화:SD05> 시켜 <골관절:LB18> 염 의 조절 기전 을 <규명:OB02> 함 . 3 . 흉 선 세포 의 발달 과정 내 에서 NS r p 7 의 <생리학적:LB07> 역할 및 기전 연구 를 위해 흉 선 세포 특이 적 <CCDC55:LC02> 유전자 제거 쥐 모델 을 <확립:LA03> 하 여 <정상:NA10> 흉 선 세포 와 의 비교 를 통한 NS r p 70 제거 흉 선 세포 발달 <표현:NA01> 형 차이 <규명:OB02> 및 <RNAseq분석:LC01> 을 통한 NS r p 70 에 의해 조절 받 는 유전자 네트워크 분석 및 흉 선 세포 <표현:NA01> 형 과 의 <관계:EE02> 성 <규명:OB02> 함 . 4 . 유전 성 질환 에서 RNA spl ici ng 단백질 네트워크 구축 하 기 위해 <SR:NB06> protein 과 heter ogenous nuclear r ib on ucle ar protein <hn:LA01> R NP 을 대상 으로 SM N 2 spl ici ng 조절 유전자 를 스크리닝 하 고 발굴 된 유전자 들 의 SM N 2 spl ici ng 조절 역할 을 <규명:OB02> 함 [SEP]'

In [24]:
df_pred

,input,encode_input,target,predict
0,1. 정전기적 농축법 개발을 통한 구형인공 지질막의 농축 및 이를 활용한 정수 시스...,[ 202 219 216 14091 3686 2554 10858 36...,"[0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",[ 0 4 4 4 4 4 4 4 4 4 4 ...
1,"자연선택과 돌연변이, 유전자재조합과 삽입삭제 등을 통한 미생물의 숙주 내 진화와 아...",[ 202 6509 6121 435 11191 2481 215 62...,"[0, 307, 308, 4, 355, 4, 4, 4, 4, 4, 4, 4, 4, ...",[ 0 4 4 4 355 4 4 4 4 4 4 ...
2,자궁 내 균이 생식세포에 미치는 영향 분석 황백 및 환삼덩굴 추출물의 유효성분 분...,[ 202 12873 736 525 2481 13308 6119 22...,"[0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 191, 192, 4, 4,...",[ 0 4 4 4 4 4 4 4 4 4 191 1...
3,2차년도 고 오메가3과 저 오메가3 지방산 생산 들깨 자원의 종자 전사체 대량 비교...,[ 202 220 6827 1007 416 2314 3985 37...,"[0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",[ 0 4 4 4 4 4 4 4 4 4 4 ...
4,"육류식이패턴 산모에서 태어난 정상 아이, 아토피피부염 아이, 그리고 건강식이패턴 산...",[ 202 2427 3867 9808 10168 1856 3832 55...,"[0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 525, 87, 4, 67,...",[ 0 4 4 4 4 4 4 4 4 4 525 ...
...,...,...,...,...
54314,FreSH tracing assay 기반 줄기세포 품질 평가법 검증 및 시스템 제작...,[ 202 11546 3633 3659 3647 8947 9577 36...,"[0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, ...",[ 0 4 4 4 4 4 4 4 4 4 4 ...
54315,"o 소형 드론 플랫폼 개발 현장 촬영과 자율 비행을 위한 카메라, 소형 짐발 및 장...",[ 202 279 8049 13777 7401 5559 6286 84...,"[0, 4, 4, 123, 124, 4, 4, 4, 4, 109, 110, 4, 4...",[ 0 4 4 123 124 4 4 4 4 109 110 ...
54316,Auto Focusing HW 개발 50mms 이상의 sacn 속도를 가지는 광학...,[ 202 13519 3617 239 8592 5537 14038 55...,"[0, 4, 4, 4, 4, 4, 123, 124, 4, 4, 4, 4, 4, 4,...",[ 0 4 4 4 4 4 123 124 4 4 4 ...
54317,종축집단 개체 증식25두 및 특성평가62두 8개 항목 모색 형질고정용 씨수소 능력검...,[ 202 2607 3901 6305 7541 9382 6071 10...,"[0, 69, 70, 4, 375, 376, 4, 4, 4, 4, 4, 4, 4, ...",[ 0 69 70 4 375 376 4 4 4 4 4 ...


In [29]:
df_data = pd.read_csv('NER_ids_label_rev_index_1112.csv')

In [30]:
df_data

,summary,NER_ids,NER_label
0,국내에서의 연구계획과 영국에서의 연구계획은 유기적으로 동시에 진행될 계획이다. 국내...,[ 0 199 4 4 175 4 4 479 4 4 175 ...,"['B▁LC13', 'O', 'O', 'B▁SH01', 'O', 'O', 'B▁OC..."
1,"본 연구의 연구내용은 다음과 같다. 첫째, 친환경농산물 급식 수요 증대에 대응한 친...",[ 0 4 175 4 4 4 4 4 4 4 4 ...,"['O', 'B▁SH01', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,* 주관기관 삼오씨엔에스 수집계층 표준 모델 Meta 관리 모듈개발 표준 모델 정보...,[ 0 4 4 4 4 4 4 223 224 505 293 2...,"['O', 'O', 'O', 'O', 'O', 'O', 'B▁LB18', 'I▁LB..."
3,아래와 같이 HW와 SW 분야로 나뉘어 집니다. HW 고음의 방향을 진동으로 알려주...,[ 0 4 4 4 411 4 399 387 4 4 4 ...,"['O', 'O', 'O', 'B▁ED08', 'O', 'B▁LC03', 'B▁HE..."
4,3차년도 신호처리 알고리즘 개발 유컴테크놀러지 항목 개발 내용 레이더 신호처리 측...,[ 0 4 4 4 581 582 615 4 4 273 4 ...,"['O', 'O', 'O', 'B▁LA07', 'I▁LA07', 'B▁SF01', ..."
...,...,...,...
49751,기존 제품의 온수커튼식 또는 다겹보온커튼식 제품은 빛의 투과를 방해하고 설치 비용이...,[ 0 625 379 4 179 180 75 76 4 4 75 ...,"['B▁ED10', 'B▁NC02', 'O', 'B▁LC05', 'I▁LC05', ..."
49752,1단계 2D transition metal dichalcogenides TMD 하부...,[ 0 4 293 173 174 4 4 4 4 4 4 ...,"['O', 'B▁SC10', 'B▁NB06', 'I▁NB06', 'O', 'O', ..."
49753,3세부 연구내용1단계 1차년도 기술 이전 1세부에서 개발한 기술의 유효성안정성 평가...,[ 0 4 4 175 4 4 293 4 4 4 609 6...,"['O', 'O', 'B▁SH01', 'O', 'O', 'B▁SC10', 'O', ..."
49754,산 첨가 및 고온 가열 살균을 통한 죽순 상온 유통 장기 저장 가능 가공 기술을 개...,[ 0 4 173 4 21 139 140 4 4 223 224 1...,"['O', 'B▁NB06', 'O', 'B▁NC08', 'B▁LB17', 'I▁LB..."


In [27]:
df_data['NER_ids'][0][1:-1]

'  0   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4\n   4   4   4   4   4   4   4   4   4   4 151   4 441   4   4   4  19   4\n   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4\n   4   4   4   4   4   4   4   4 453   4   4   4   4   4   4   4   4   4\n   4 437 438   4 603   4   4   4   4   4   4   4   4   4   4   4   4   4\n   4   4   4   4 511   4   4   4   4   4   4   4   4   4   4   4   4   4\n   4 453   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4\n 441   4   4   4   4   4   4   4 139 140   4   4   4   4   4   4   4   4\n   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4\n   4   4   4   4   4   4   4 511   4   4   4   4   4   4   4   4   4   4\n   4   4   4 193   4   4   4   4   4   4   4   4   4 139 140 511   4   4\n   4   4   4   4   4   4   4   4   4   4   4   4 139 140   4   4 511   4\n   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4\n   4   4   4   4   4   4   4   4   4  

In [31]:
i = 23535
mecab_summary = mecab.morphs(df_data['summary'].iloc[i])
enc_input = tokenizer.encode(' '.join(mecab_summary))
pred = df_data['NER_ids'][i][1:-1].split()
(pred_to_text(enc_input, pred))

'[CLS] <클라우드서버:EA15> <연동:HB05> <인터렉션:ND08> <광고용:SF06> <디지털사이니지:HE04> <단말기:HD09> 기 및 <웹뷰:HB02> <기반:HB02> 의 <인터렉션:ND08> <광고:NC08> <플랫폼개발:SD03> 2 차년 도 <클라우드서버:EA15> <연동:HB05> <인터렉션:ND08> <광고:NC08> 용 <안드로이드:NC08> <디지털사이니지:HE04> <단말기:HD09> 기 <개발:EH05> <클라우드서버:EA15> <연동:HB05> <인터렉션:ND08> <광고:NC08> 용 <웹뷰:HB02> <기반:HB02> <인터렉션:ND08> <광고:NC08> <솔루션개발:EE04> <클라우드서버:EA15> <연동:HB05> <인터렉션:ND08> <광고:NC08> 용 <디지털사이니지:HE04> <시제품제작:LA09> 및 <인터렉션:ND08> <광고:NC08> <플랫폼:HD08> <연동:HB05> <시연:EC03> , <검증:HE02> <특허:NC08> <신규:NA09> 출원 2 건 <이상:ED07> <시제품:NC01> <클라우드서버:EA15> <연동:HB05> <인터렉션:ND08> <광고:NC08> 용 <디지털사이니지:HE04> <단말기:HD09> 기 <클라우드서버:EA15> <연동:HB05> <인터렉션:ND08> <광고:NC08> 용 <웹뷰:HB02> <기반:HB02> <인터렉션:ND08> <광고:NC08> <솔루션:NC08> <웹뷰:HB02> <기반:HB02> <HTML5:EC10> <인터렉션:ND08> <광고:NC08> , <인터렉션:ND08> <광고:NC08> <컨텐츠:EG05> <저작:HE07> 툴 [SEP]'

In [28]:
i = 23535
mecab_summary = mecab.morphs(df_data['summary'].iloc[i])
enc_input = tokenizer.encode(' '.join(mecab_summary))
pred = df_data['NER_ids'][i][1:-1].split()
(pred_to_text(enc_input, pred))

'[CLS] 선정 된 <지역특산물:HE14> , 과일 채 소 류 의 생리활성 <에:SG06> 적합 <한기능성식품:HE14> 개발 5 종 제품 <의:HE12> 선호도 평가 및 시장 성 <조사:EE03> 개별 인 정 형 <원료:HB03> 등록 을 위한 원료 표준 화 및 표준 화 된 제조공정 이용 인체 적용 시험 의 시험 대조 식품 생산 [SEP]'

In [181]:
pred_label_texts = []
for i in tqdm(range(len(df_pred))):
    mecab_input = mecab.morphs(df_pred['input'].iloc[i])
    enc_input = tokenizer.encode(mecab_input)
#     enc_input = (df_pred['encode_input'].iloc[i][1:-1].replace(',', '').split())
    pred = df_pred['predict'].iloc[i][1:-1].replace(',', '').split()
    pred_label_texts.append(pred_to_text(enc_input, pred))

100%|██████████| 37013/37013 [01:24<00:00, 435.96it/s]


In [182]:
target_label_texts = []
for i in tqdm(range(len(df_pred))):
    mecab_input = mecab.morphs(df_pred['input'].iloc[i])
    enc_input = tokenizer.encode(mecab_input)
#     enc_input = (df_pred['encode_input'].iloc[i][1:-1].replace(',', '').split())
    pred = df_pred['target'].iloc[i][1:-1].replace(',', '').split()
    target_label_texts.append(pred_to_text(enc_input, pred))

100%|██████████| 37013/37013 [01:46<00:00, 348.32it/s]


In [10]:
# label_texts = []
# for i in tqdm(range(len(df_pred))):
#     enc_input = tokenizer.encode(df_pred['input'].iloc[i])[1:-1]
#     pred = df_pred['predict'].iloc[i][1:-1].replace(',', '').split()
#     label_texts.append(pred_to_BI_text(enc_input, pred))

In [198]:
pred[1:-1]

['4',
 '4',
 '543',
 '544',
 '544',
 '544',
 '544',
 '4',
 '543',
 '544',
 '544',
 '544',
 '544',
 '544',
 '543',
 '543',
 '544',
 '544',
 '4',
 '4',
 '4',
 '4',
 '4',
 '19',
 '20',
 '20',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '213',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '543',
 '544',
 '544',
 '4',
 '4',
 '4',
 '543',
 '544',
 '4',
 '4',
 '4',
 '535',
 '4',
 '511',
 '4',
 '475',
 '476',
 '476',
 '501',
 '502',
 '502',
 '4',
 '4',
 '4',
 '543',
 '544',
 '544',
 '4',
 '4',
 '4',
 '4',
 '4',
 '443',
 '4',
 '4',
 '4',
 '4',
 '543',
 '4',
 '543',
 '544',
 '544',
 '4',
 '4',
 '4',
 '4',
 '101',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '19',
 '4',
 '4',
 '543',
 '4',
 '543',
 '544',
 '544',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '19',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '543',
 '544',
 '544',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '543',
 '544',
 '544',
 '4',
 '4',
 '4',
 '4',
 '325',
 '4',
 '4',
 '543',
 

In [199]:
i = 35
enc_input = tokenizer.encode(df_pred['input'].iloc[i])
pred = df_pred['target'].iloc[i][1:-1].replace(',', '').split()
(pred_to_text(enc_input, pred))
# enc_input[1:-1]

'[CLS] 1. <역분화:LA03> 로 <부터내:LA03> <세포:LA03> <주확립:LA03> 배엽전구세포 <분화후:LC01> CXCR4 & <CD:EA05> 117 동시에 양성인 분획을 FACS so <rting:LA03> 내배엽 <전구:LA03> 세포주 배양 <조건:LA10> 에 <서:LC07> 세포주 <배양가능:LA04> <확인및:LC02> 적인 계 <대배양:LA03> 에도 내배엽전 <구:LC10> 세포주의 특성 유지 <여부:LA03> 조사 <2.:LA03> 된 내배엽 <전:LA06> 구세포주를 in vivo 이식 <후:LC01> , te <ra:LA03> to <ma형성:LA03> 검증 확립된 내배엽전구세포주를 면역결핍 마우스 <에:LC01> 이식후, terato <ma형성:LA03> 조사 생성된 조직에서 내 <배엽:LA03> 항체로 면역염색 <을:LB17> 통해 내 <배엽:LA03> 의 <세포분화:LA03> 여부 조사 외 <배:LB17> 엽 <이:LC07> 나 중 <배엽:LA03> 로의 분화 유도 여부 확인 <3.:LC03> 배엽전 <구세포:LC03> 를 이용한 인슐린 분비세포로의 분화 인슐린 분비세포로의 <분화시:LC06> , 세포의 <분화에:LA03> 조사 <및효율적:LB17> 인 <프로토콜개발:LC03> 분화 <만능:LA03> 로부터 직접분화 유도한 <인슐린:LB17> 분비세포와 내배엽전구세포주에서의 분화 <유도간:LC03>'

In [193]:
# 34
(target_label_texts[35])

'[CLS] 1. <[UNK][UNK]:LA03> [UNK] <분화후:LA03> <CD:LA03> <[UNK]동시:LA03> 양성 인 분획 을 FACS <[UNK]및:LC01> [UNK] 배양 조건 에서 세포주 배양 <가능:EA05> 여부 확인 및 지속 적 인 [UNK] 이후 에 도 [UNK] <[UNK]의:LA03> 유지 여부 조사 <2.:LA03> 확립 된 [UNK] <[UNK]:LA10> 를 <in:LC07> vivo <이식후:LA04> <[UNK]형성:LC02> 검증 확립 된 <[UNK][UNK]:LA03> [UNK] 마우스 에 [UNK] [UNK] <[UNK]:LC10> 형성 여부 조사 생성 <된:LA03> 조직 <에서[UNK]:LA03> 항체 로 [UNK] 을 <통해:LA06> [UNK] 계열 의 세포 분화 유도 여부 <조사:LC01> [UNK] [UNK] <[UNK]:LA03> 으로 <의분화:LA03> 여부 확인 3. [UNK] [UNK] 를 이용 한 인슐린 [UNK] 로 의 분화 인슐린 <[UNK]:LC01> 로 의 분화 시 [UNK] 세포 <의분화:LA03> 필요 한 신호전달 조사 및 효율 적 <인프로토콜:LA03> [UNK] [UNK] 직접 분화 <유도:LB17> 한 인슐린 <[UNK]와:LA03> [UNK] <에서의:LA03> 유도 간 의 <효율:LB17> 성 <비교:LC07> 4. <췌도:LA03> 배양 을 이용 한 인슐린 [UNK] <의분화:LC03> 성 증대 분화 <단계에서:LC03> [UNK] [UNK] [UNK] [UNK] 와 성숙 한 [UNK] 공동 배양 을 통해 인슐린 <[UNK]로:LC06> 의 분화 효율 <성검증:LA03> 3 <D배양에:LB17> 의한 <분화비교:LC03> 통해 <향상된:LA03> [UNK] 로 의 분화 여부 를 <평가:LB17> 최종 분화 유도 된 인슐린 [UNK] 가 in vitro 상 에서 포도당 자극 시 <[UNK]인슐린:LC03>'

In [173]:
(target_label_texts[35])

'[CLS] 1. <[UNK]로부터:LA03> [UNK] <분화후:LA03> <CD:LA03> <[UNK]동시:LA03> 양성 인 분획 을 [UNK] <[UNK]및:LC01> [UNK] 배양 조건 에서 세포주 배양 <가능:EA05> 여부 확인 및 지속 적 인 [UNK] 이후 에 도 [UNK] <[UNK]의:LA03> 유지 여부 조사 <2.:LA03> 확립 된 [UNK] <[UNK]:LA10> 를 <in:LC07> vivo <이식후:LA04> <[UNK]형성:LC02> 검증 확립 된 <[UNK][UNK]:LA03> [UNK] 마우스 에 [UNK], <[UNK]:LC10> 형성 여부 조사 생성 <된:LA03> 조직 <에서[UNK]:LA03> 항체 로 [UNK] 을 <통해:LA06> [UNK] 계열 의 세포 분화 유도 여부 <조사:LC01> [UNK] 이나 <[UNK]:LA03> 으로 <의분화:LA03> 여부 확인 3. [UNK] [UNK] 를 이용 한 인슐린 [UNK] 로 의 분화 인슐린 <[UNK]:LC01> 로 의 분화 시, 세포 <의분화:LA03> 필요 한 신호전달 조사 및 효율 적 <인프로토콜:LA03> [UNK] 로부터 직접 분화 <유도:LB17> 한 인슐린 <[UNK]와:LA03> [UNK] <에서의:LA03> 유도 간 의 <효율:LB17> 성 <비교:LC07> 4. <췌도:LA03> 배양 을 이용 한 인슐린 [UNK] <의분화:LC03> 성 증대 분화 <단계에서:LC03> [UNK] [UNK] [UNK] [UNK] 와 성숙 한 [UNK] 공동 배양 을 통해 인슐린 <[UNK]로:LC06> 의 분화 효율 <성검증:LA03> 3 <D배양에:LB17> 의한 <분화비교:LC03> 통해 <향상된:LA03> [UNK] 로 의 분화 여부 를 <평가:LB17> 최종 분화 유도 된 인슐린 [UNK] 가 in vitro 상 에서 [UNK] 자극 시 <,인슐린:LC03>'

In [158]:
(pred_label_texts[34])
# df_pred['input'].iloc[30]

'[CLS] 모바일 기기 지원영역 확대 <i:LB06> OS, Windows 최신 Android 버전을 지원하는 Image 개발 모바일 DLP 기능 강화 개발 기술 최적화 및 고도화 능동형 프로토콜 분석기술 개발 후 <처리기술:LA02> 화 상품화를 위한 지원영역 확대 및 최적화 SecaaS 프레임워크의 상품화를 위한 고도화 및 테스트 [SEP]'

In [170]:
df_pred['predict'].iloc[34][1:-1].replace(',', '').split()
df_pred['encode_input'].iloc[34]

'[  202  7205  7432   201  6543   201   215   201 11452   201 14283  2446\n  6010  3397   889   201  5559  7205   201  5737  6612  5559  5579  5922\n  3495  1576  8067  3495   201  8231  5578  5579  5559   201  5579  8067\n  3495  7340  3495  1374  5611   201  6543  1576  5922  3495   201 11875\n  2464  7340  3495  1374  5611  8067  3495  1576  7517   203   200   200\n   200   200   200   200   200   200   200   200   200   200   200   200\n   200   200   200   200   200   200   200   200   200   200   200   200\n   200   200   200   200   200   200   200   200   200   200   200   200\n   200   200   200   200   200   200   200   200   200   200   200   200\n   200   200   200   200   200   200   200   200   200   200   200   200\n   200   200   200   200   200   200   200   200   200   200   200   200\n   200   200   200   200   200   200   200   200   200   200   200   200\n   200   200   200   200   200   200   200   200   200   200   200   200\n   200   200   200   200   200   200 

In [164]:
(df_pred['target'].iloc[34])

'[0, 4, 4, 4, 4, 369, 4, 4, 4, 4, 4, 371, 372, 4, 4, 4, 4, 4, 4, 19, 4, 4, 4, 591, 592, 265, 266, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 335, 336, 336, 569, 570, 570, 4, 4, 4, 4, 4, 4, 4, 4, 369, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]'

In [ ]:
# 245,12344
df_pred['input'].iloc[7777]

In [111]:
pred_label_texts[7777]

'[CLS] <유연:NC09> <촉각센서신호처리:OB01> <기술개발촉각센서어레이:NC08> 를 위한 <센서:SI01> <네트워크:SI01> <기술개발유연:NC09> <촉각센서:NC08> 응용을 위한 <로봇:SH07> <플랫폼:SI05> <개발:SF01> <유연:NC09> <촉각센서:NC08> 성능평가를 위한 <그리퍼제작:SI02> 및 <테스트베드:SC15> <구축:SC07> <유연:NC09> 3D회로기술개발 <하이브리드:SE06> 첨가성형 <공정계획:EI02> 및 <제어기술:LC11> <개발:SF01> <탄성:NC05> <재료:SE05> 첨가 <가공:SI05> <기술개발하이브리드:SE06> 첨가성형장치 <개발:SF01> 신축성30 % 를갖는광경화성피복소재개발 <탄소나노튜브:NC10> [UNK] <그래핀:OB01> [UNK] <금속:ND03> 나노소재를이용한신축성30 % [UNK] 감도0. 1N을갖는센서소재개발 [SEP]'

In [112]:
target_label_texts[7777]

'[CLS] <유연:NC09> <촉각센서신호처리:OB01> <기술개발촉각센서어레이:NC08> 를 위한 <센서:SI01> <네트워크:SI01> <기술개발유연:NC09> <촉각센서:NC08> 응용을 위한 <로봇:SH07> <플랫폼:SI05> <개발:SF01> <유연:NC09> <촉각센서:NC08> 성능평가를 위한 <그리퍼제작:SI02> 및 <테스트베드:SC15> <구축:SC07> <유연:NC09> 3D회로기술개발 <하이브리드:SE06> 첨가성형 <공정계획:EI02> 및 <제어기술:LC11> <개발:SF01> <탄성:NC05> <재료:SE05> 첨가 <가공:SI05> <기술개발하이브리드:SE06> 첨가성형장치 <개발:SF01> <신축성:NC08> 30 % 를갖는광경화성피복소재개발 <탄소나노튜브:NC10> [UNK] <그래핀:OB01> [UNK] <금속:ND03> 나노소재를이용한신축성30 % [UNK] <감도:LC04> 0. 1N을갖는센서소재개발 [SEP]'

In [26]:
tokenizer.tokenize(df_pred['input'].iloc[200])

['융합',
 '##공정의',
 '효율성',
 '평가',
 '및',
 '기본설계',
 '##안',
 '도출',
 '열변형',
 '##파',
 '##쇄',
 '선별',
 '기반',
 '고도',
 '##토양',
 '##세척',
 '##기술',
 '[UNK]',
 'H',
 '##SF',
 '##기반',
 '고도',
 '##지',
 '##중',
 '##정화',
 '##기술에',
 '대한',
 '현장적용',
 '##성',
 '최종',
 '##평가',
 '기본설계',
 '##안',
 '및',
 '매뉴얼',
 '작성',
 '토양',
 '##환경',
 '##보전',
 '##법',
 '지',
 '##목',
 '##별',
 '우려',
 '##기준',
 '만족',
 'Ex',
 '##s',
 '##itu',
 '열변형',
 '##파',
 '##쇄',
 '선별',
 '기반',
 '고도',
 '##토양',
 '##세척',
 '##공정',
 '효율',
 '##평가',
 'Ins',
 '##itu',
 'H',
 '##SF',
 '기반',
 '고도',
 '##지',
 '##중',
 '##정화',
 '##공정',
 '효율',
 '##평가',
 '3차년도',
 '현장실증',
 '##실험',
 '결과에',
 '##서',
 '도출한',
 '최적',
 '##인자를',
 '바탕으로',
 'Pilot',
 '장치',
 '보완',
 '융합',
 '##공정',
 '운전',
 '##인자',
 '결정',
 '[UNK]',
 '성능평가',
 '후',
 '설계',
 '##보완',
 '정화',
 '##효율',
 '현장',
 '##검증']

In [36]:
df_pred['target'].iloc[200]

'[0, 563, 4, 463, 291, 4, 465, 4, 4, 507, 4, 4, 265, 217, 391, 4, 4, 4, 4, 4, 4, 4, 391, 4, 4, 4, 4, 4, 19, 20, 4, 4, 465, 4, 4, 297, 4, 471, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 507, 4, 4, 265, 217, 391, 4, 4, 4, 559, 4, 4, 4, 4, 4, 217, 391, 4, 4, 4, 4, 559, 4, 4, 233, 234, 4, 4, 4, 539, 4, 4, 4, 441, 4, 563, 4, 337, 4, 507, 4, 393, 4, 69, 4, 337, 4, 371, 4, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]'

In [39]:
df_pred['target'].iloc[200][2:-2].replace(',', '').split()

['563',
 '4',
 '463',
 '291',
 '4',
 '465',
 '4',
 '4',
 '507',
 '4',
 '4',
 '265',
 '217',
 '391',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '391',
 '4',
 '4',
 '4',
 '4',
 '4',
 '19',
 '20',
 '4',
 '4',
 '465',
 '4',
 '4',
 '297',
 '4',
 '471',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '4',
 '507',
 '4',
 '4',
 '265',
 '217',
 '391',
 '4',
 '4',
 '4',
 '559',
 '4',
 '4',
 '4',
 '4',
 '4',
 '217',
 '391',
 '4',
 '4',
 '4',
 '4',
 '559',
 '4',
 '4',
 '233',
 '234',
 '4',
 '4',
 '4',
 '539',
 '4',
 '4',
 '4',
 '441',
 '4',
 '563',
 '4',
 '337',
 '4',
 '507',
 '4',
 '393',
 '4',
 '69',
 '4',
 '337',
 '4',
 '371',
 '4',
 '1',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 '2',
 